Integrantes: Yoselin Araceli Campos Sacramento, Ana Margarita Reyes Hernandez, Alexis Martinez Arteaga

Importamos las librerias necesarias

In [2]:
import pandas as pd, time
import dask.dataframe as ddf
import os

Creamos dos arreglos. El primero servira para guardar los nombres de los archivos a evaluar. El segundo almacenara el nombre correcto de todas las columnas de un DataFrame.

In [3]:
datos_csv = []

In [4]:
lista_diferencias_ttdf_diferencia = []

In [5]:
lista_viajes_largos = []

Funcion que permite leer un archivo .csv y convertirlo en un DataFrame

In [6]:
def lectura_csv():
    archivos_csv = 'archivos_csv/trip_data_{:02d}.csv'
    datos = (archivos_csv.format(k) 
             for k in range(1,13))
    for dato in datos:
        datos_csv.append(dato)

In [7]:
lectura_csv()

In [20]:
def huella_de_memoria():
  '''Regresa la momoria usada por el proceso de python'''
  mem = psutil.Process(os.getpid()).memory_info().rss
  return (mem / 1024**2)

In [ ]:
def todos_pandas():
    for i in range(0, 12):
        huella_inicio = huella_de_memoria()
        huella_inicio
        t_start = time.time()
        df = pd.read_csv(datos_csv[i])
        t_end = time.time()
        print('Tiempo que tarda pandas en construir un DataFrame: {} s'.format(t_end-t_start))
        huella_fin = huella_de_memoria()
        huella_fin
        print("La huella de memoria utilizada para el proceso fue: {} mb".format(huella_fin - huella_inicio))
        df.isna().sum()
        df_nulos_p1 = df[df['store_and_fwd_flag'].isna()]
        df_nulos_p2 = df[df['dropoff_longitude'].isna()]
        df_nulos_p3 = df[df['dropoff_latitude'].isna()]
        df = df.dropna(how='all', subset=['dropoff_longitude', 'dropoff_latitude'])
        df = df.dropna(how='all', subset=['store_and_fwd_flag'])
        df.isna().sum()
        td_dife_cero = (df['trip_distance'] != 0.0)
        inicio = time.time()
        td_dife_cero.mean()
        fin = time.time()
        print('Tiempo que tarda pandas en optener el promedio de la distancia de viaje: {} s'.format(fin-inicio))
        df.shape

In [ ]:
todo_pandas()

6.3 - Funcion que permite obtener el promedio de trip_distance en un DataFrame

In [8]:
def promedio_trip_distance(data_frame):
    td_dask = data_frame['trip_distance']
    t_inicio = time.time()
    td_dask.mean()
    t_fin = time.time()
    print("El tiempo que tarda Dask en obtener el promedio de trip_distance es: {} s".format(t_fin-t_inicio))

6.4 - Funcion que permite obtener el promedio de trip_time_in_secs en un DataFrame

In [9]:
def promedio_trip_time_in_secs(data_frame):
    ttis_dask = data_frame['trip_time_in_secs']
    tiempo_i = time.time()
    ttis_dask.mean()
    tiempo_f = time.time()
    print("El tiempo que tarda Dask en obtener el promedio de trip_time_in_secs es: {} s".format(tiempo_f-tiempo_i))

6.5 - Funcion que permite obtener la diferencia entre pickup_datetime y dropoff_datetime

In [10]:
def diferencia_pickup_dropoff(data_frame):
    data_frame['pickup_datetime'] = ddf.to_datetime(data_frame['pickup_datetime'])
    data_frame['dropoff_datetime'] = ddf.to_datetime(data_frame['dropoff_datetime'])
    duracion = data_frame['dropoff_datetime'] - data_frame['pickup_datetime']
    data_frame = data_frame.assign(Duracion = duracion)
    data_frame['Duracion'] = data_frame['Duracion'].dt.total_seconds()
    comparacion = data_frame['Duracion'] - data_frame['trip_time_in_secs']
    comparacion.astype('int64')
    v = comparacion.count().compute()
    lista_diferencias_ttdf_diferencia.append(v)

6.6 - Funcion que permite definir los viajes largos como aquellos que duran mas de 20 minutos

In [11]:
def definir_viajes_largos(data_frame):
    viajes_largos = data_frame[data_frame['trip_time_in_secs'] > 1200]
    v = viajes_largos['trip_time_in_secs'].count().compute()
    lista_viajes_largos.append(v)
    taxis_

8(a) - Numero de taxis diferentes que hiciseron un recorrido (o mas) de mas de 20 minutos

In [ ]:
def taxis_diferentes(data_frame):
    taxis = viajes_largos.groupby('medallion')
    taxis.count().compute()
    vehiculo = viajes_largos
    vehiculos = vehiculo.drop(['hack_license', 'vendor_id', 'store_and_fwd_flag',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_time_in_secs', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1)
    print("Vehiculo que mas viajes hizo al mes: ")
    vehiculos.max().compute()

Funcion principal que permite llamar a todas las funciones anteriores

In [12]:
def main():
    for i in range(0,12):
        print("archivo csv: {}".format(i))
        df_dask = ddf.read_csv(datos_csv[i], low_memory = False)
        promedio_trip_distance(df_dask)
        promedio_trip_time_in_secs(df_dask)
        diferencia_pickup_dropoff(df_dask)
        definir_viajes_largos(df_dask)

In [13]:
main()

archivo csv: 0
El tiempo que tarda Dask en obtener el promedio de trip_distance es: 0.0039942264556884766 s
El tiempo que tarda Dask en obtener el promedio de trip_time_in_secs es: 0.0 s
archivo csv: 1
El tiempo que tarda Dask en obtener el promedio de trip_distance es: 0.0 s
El tiempo que tarda Dask en obtener el promedio de trip_time_in_secs es: 0.0 s
archivo csv: 2
El tiempo que tarda Dask en obtener el promedio de trip_distance es: 0.0 s
El tiempo que tarda Dask en obtener el promedio de trip_time_in_secs es: 0.0 s
archivo csv: 3
El tiempo que tarda Dask en obtener el promedio de trip_distance es: 0.0 s
El tiempo que tarda Dask en obtener el promedio de trip_time_in_secs es: 0.015621185302734375 s
archivo csv: 4
El tiempo que tarda Dask en obtener el promedio de trip_distance es: 0.0 s
El tiempo que tarda Dask en obtener el promedio de trip_time_in_secs es: 0.0 s
archivo csv: 5
El tiempo que tarda Dask en obtener el promedio de trip_distance es: 0.0 s
El tiempo que tarda Dask en ob

6.5(b) - Numero de registros que en los que la columna trip_time_in_secs es diferente a la nueva columana de Duracion

In [17]:
contador = 0
for i in lista_diferencias_ttdf_diferencia:
    print("El archivo {} tiene {} diferencias".format((contador+1), i))
    contador=contador+1

El archivo 1 tiene 12597109 diferencias
El archivo 2 tiene 13823840 diferencias
El archivo 3 tiene 13971118 diferencias
El archivo 4 tiene 13990176 diferencias
El archivo 5 tiene 14107693 diferencias
El archivo 6 tiene 14388451 diferencias
El archivo 7 tiene 14385456 diferencias
El archivo 8 tiene 14776615 diferencias
El archivo 9 tiene 15004556 diferencias
El archivo 10 tiene 15100468 diferencias
El archivo 11 tiene 15285049 diferencias
El archivo 12 tiene 15749228 diferencias


7 - Cantidad de viajes largos

In [18]:
contador = 0
for j in lista_viajes_largos:
    print("El archivo {} tiene {} viajes que duraron mas de 20 minutos".format((contador+1), j))
    contador=contador+1

El archivo 1 tiene 1919857 viajes que duraron mas de 20 minutos
El archivo 2 tiene 2084036 viajes que duraron mas de 20 minutos
El archivo 3 tiene 2428514 viajes que duraron mas de 20 minutos
El archivo 4 tiene 1768887 viajes que duraron mas de 20 minutos
El archivo 5 tiene 2417638 viajes que duraron mas de 20 minutos
El archivo 6 tiene 2373698 viajes que duraron mas de 20 minutos
El archivo 7 tiene 2400513 viajes que duraron mas de 20 minutos
El archivo 8 tiene 1715237 viajes que duraron mas de 20 minutos
El archivo 9 tiene 2545155 viajes que duraron mas de 20 minutos
El archivo 10 tiene 2264701 viajes que duraron mas de 20 minutos
El archivo 11 tiene 2552684 viajes que duraron mas de 20 minutos
El archivo 12 tiene 2111851 viajes que duraron mas de 20 minutos
